In [1]:
import numpy as np
import pandas as pd

# You may choose to select different parameters/values
number_assets = 32
T  = 328
# Read returns
df = pd.read_csv('returns_data.txt',delim_whitespace=True)

Rraw = df.values.T

# Select the first N,T assets and scenarios, you may choose a different strategy if you would like to do so.
R = Rraw[:number_assets,:T]

# Expected return of each asset
expected_returns = np.mean(R, axis = 1)

# Covariance matrix of asset returns
covariance_matrix = np.cov(R)

In [2]:
from pyqubo import Array

D = 20

x = Array.create('x', shape=(number_assets, D), vartype="BINARY")
y = Array.create('y', shape=T, vartype="BINARY")
s = Array.create('s', shape=(T+1, D), vartype="BINARY")
r_e = Array.create('r_e', shape=D, vartype="BINARY")

In [3]:
def binary_encode(x, D):
    bin_x = 0
    for d in range(D):
        bin_x += (1/2**(D-1))*(2**d)*x[d]
    return bin_x

def encode_s(s, D):
    bin_s = 0
    for d in range(D):
        bin_s += (2**d)*s[d]
    return bin_s
    #return LogEncInteger(s)

def encode_r_e(r_e, z, Z):
    bin_r = 0
    for d in range(D):
        bin_r += (1/2**(D-1))*(2**d)*r_e[d]
    return -z+(Z+z)*bin_r

In [4]:
def H1(covariance_matrix, x):
    ham_1 = 0
    n, D = x.shape
    for i in range(n):
        x_i = binary_encode(x[i], D)
        for j in range(n):
            x_j = binary_encode(x[j], D)
            ham_1 += covariance_matrix[i, j]*x_i*x_j
    return ham_1

In [5]:
def H2(expected_returns, x):
    ham_2 = 0
    n, D = x.shape
    for i in range(n):
        ham_2 -= expected_returns[i]*binary_encode(x[i], D)
    return ham_2

In [6]:
def H3_t(r_e, x, R, M, y, s, z, Z, t):
    ham_3 = encode_r_e(r_e, z, Z)
    n, D = x.shape
    for i in range(n):
        ham_3 -= R[i, t]*binary_encode(x[i], D)
    ham_3 += M*(1-y[t])
    ham_3 += encode_s(s[t], D)
    return ham_3

def H3(r_e, x, R, M, y, s, z, Z):
    ham_3 = 0
    for t in range(T):
        ham_3 += (H3_t(r_e, x, R, M, y, s, z, Z, t))**2
    return ham_3

In [7]:
def H4(y, T, s, epsilon, D):
    ham_4 = 0
    for t in range(T):
        ham_4 += y[t]
    ham_4 -= (1-epsilon)*T
    ham_4 -= encode_s(s[-1], D)
    return ham_4**2

In [8]:
def H5(x):
    ham_5 = 0
    n, D = x.shape
    for i in range(n):
        ham_5 += binary_encode(x[i], D)
    return (ham_5-1)**2

In [9]:
lambda_ = [1, 1, 1, 1, 1]
M = 0.005758011095403033
z = -0.01788126470126068
Z = 0.004813770295956064
epsilon = 0.05
H = lambda_[0]*H1(covariance_matrix, x)+lambda_[1]*H2(expected_returns, x)+lambda_[2]*H3(r_e, x, R, M, y, s, z, Z)+lambda_[3]*H4(y, T, s, epsilon, D)+lambda_[4]*H5(x)

<div class="alert alert-block alert-danger">
    
<b>BONUS EXERCISE:</b> 
As you notice the number of binary operators one has to use scales with the number of variables. Introducing the slack variables is therefore very costly. Try to suggest a new way of implementing the inequalities, without a lot of details. You can use the help of reference [Unbalanced penalization: A new approach to encode inequality constraints of combinatorial problems for quantum optimization algorithms](https://arxiv.org/abs/2211.13914).

</div>

**Answer:**

Optimization problems, like the one above, can be solved with the use of simulated annealing. It is one of the most preferred heuristic methods to solve optimization problems, since it avoids local minima.

In [10]:
# write your code here
model = H.compile()

bqm = model.to_bqm()
#import neal
#sa = neal.SimulatedAnnealingSampler()
#sampleset = sa.sample(bqm, num_reads=10)
#decoded_samples = model.decode_sampleset(sampleset)
#best_sample = min(decoded_samples, key=lambda x: x.energy)
#print(best_sample.sample)

In [11]:
(T+number_assets+2)*D+T

7568

In [12]:
from dwave.system import LeapHybridSampler
import numpy as np

result = LeapHybridSampler().sample(bqm, label='Notebook - Hybrid Computing 1')
print("Found solution with {} nodes at energy {}.".format(np.sum(result.record.sample), 
                                                          result.first.energy))

Found solution with 461 nodes at energy 0.2579403104997482.


In [ ]:
7568

In [12]:
def calculate_historical_VaR(weights, mu_R, confidence_level=0.95):
    
    portfolio_returns = np.dot(mu_R, np.transpose(np.array(weights)))

    VaR = np.percentile(portfolio_returns, 100 * (1 - confidence_level))
    
    return -VaR

with $\textit{weights}$ being a list that includes the optimal values (your solution) for the variables $\{x_{i}\}$ and $\textit{mu_R}$ the expected returns of the assets (given by the data). Compare the return value got from this function to $VaR_{\epsilon}$ calculated above.

In [13]:
# write your code here

def cal_asset_i(dict, i, D):
    x_i = 0
    for d in range(D):
        x_i += (1/2**(D-1))*2**(d)*dict[f"x[{i}][{d}]"]
    return x_i

def cal_r_e(dict, D, z, Z):
    r_ = 0
    for d in range(D):
        r_ += (1/2**(D-1))*2**(d)*dict[f"r_e[{d}]"]
    return -z+(Z+z)*r_

In [14]:
def expected_total_returns(expected_returns, number_assets, dict, D):
    total_return = 0
    for i in range(number_assets):
        total_return += expected_returns[i]*cal_asset_i(dict, i, D)
    return total_return

In [15]:
def variance(dict, covariance_matrix, number_assets, D):
    var = 0
    for i in range(number_assets):
        for j in range(number_assets):
            var += covariance_matrix[i, j]*cal_asset_i(dict, i, D)*cal_asset_i(dict, j, D)
    return var

In [16]:
expected_total_returns(expected_returns, number_assets, best_sample.sample, D)

0.01602504618427734

In [17]:
w_ = [cal_asset_i(best_sample.sample, i, D) for i in range(number_assets)]

calculate_historical_VaR(w_, expected_total_returns(expected_returns, number_assets,  best_sample.sample, D))

-0.0

**Answer:**

### Section 5: Quantum optimization (Quantum Annealing and/or QAOA)


**Note**: For the exercises below use the $\hat{H}$ you found in **Section 2**, with the same encoding you used in that exercise and with the use of the **data** provided in the beginning of the challenge.

**STEP 1:** Study the **Mean-Variance-VaR Simplified Version** problem with QAOA and Quantum Annealing. 

The Quantum Approximate Optimization Algorithm (QAOA) was first introduced in [A Quantum Approximate Optimization Algorithm](https://arxiv.org/abs/1411.4028). QAOA is a popular method to solve combinatorial optimization problems in noisy intermediate-scale quantum (NISQ) devices.

Useful tutorials on implementing QAOA can be found here: 
- [Pulser tutorial QAOA](https://pulser.readthedocs.io/en/latest/tutorials/qubo.html)
- [Qiskit QAOA](https://docs.quantum.ibm.com/api/qiskit/qiskit.algorithms.minimum_eigensolvers.QAOA), [Qiskit tutorial QAOA](https://qiskit.org/documentation/stable/0.40/tutorials/algorithms/05_qaoa.html)
- [pyQAOA tutorial QAOA](https://grove-docs.readthedocs.io/en/latest/qaoa.html)
- [PennyLane tutorial QAOA](https://pennylane.ai/qml/demos/tutorial_qaoa_intro/)
- [OpenQAOA-EntropicaLabs](https://openqaoa.entropicalabs.com/)

For Quantum Annealing, you may read this reference [D-Wave, Quantum Annealing](https://docs.dwavesys.com/docs/latest/c_gs_2.html#getting-started-qa) and follow [D-Wave Ocean Software documentation](https://docs.ocean.dwavesys.com/en/stable/index.html) for the implementation. 
In [Solving the Optimal Trading Trajectory Problem Using a Quantum Annealer](https://arxiv.org/abs/1508.06182), the authors explain how the choice of encoding might differ when considering solving an optimization problem with quantum annealing instead of simulated annealing.
   
Do not implement anything at the moment. This step is for introducing you to different quantum or quantum-inspired algorithms one could utilize for this problem.

**STEP 2:** In **Section 3** you used [D-Wave neal](https://docs.ocean.dwavesys.com/projects/neal/en/latest/) to solve the problem with Simulated Annealing. As also seen in the previous step, [D-Wave Ocean](https://docs.ocean.dwavesys.com/en/stable/getting_started.html)  gives you immediate, secure access to D-Wave quantum and hybrid solvers. Study the capabilities of [D-Wave Ocean](https://docs.ocean.dwavesys.com/en/stable/getting_started.html) and answer the following questions:

- For reasonable granularity (i.e. number of qubits per encoded variable) how many assets $n$ and events $T$ you can study with a quantum device and D-Wave? For that you will need to find the total number of qubits that you can explore with [D-Wave Ocean](https://docs.ocean.dwavesys.com/en/stable/getting_started.html) for the algorithm of your choice. Notice that the resources available to the public are much less than the capabilities of these devices.

**STEP 3:** After exploring the capabilities of [D-Wave Ocean Software documentation](https://docs.ocean.dwavesys.com/en/stable/index.html), you are encouraged to use a **quantum simulator/backend** to study the **Mean-Variance-VaR Simplified Version** problem with the **data** provided in the beginning of the challenge (feel free to change $T$ and $n$ if needed)  and any method of your choice. For this, you can use your qBraid account. Explore the options below:

- [Qiskit](https://docs.quantum.ibm.com/api/qiskit)
- [Amazon Braket](https://docs.aws.amazon.com/braket/latest/developerguide/what-is-braket.html)

Or even more, you may also opt to explore the usage of GPUs for simulating quantum algorithms. For example, see a recent [work](https://www.jpmorgan.com/technology/technology-blog/quantum-optimization-research) that studies QAOA scaling performance on a fast GPU-specific QAOA simulator.

No matter what algorithm you choose, **you are encouraged to run small-scale simulations of the problems on quantum backends and simulators, always keeping in mind the resources that you are utilizing and their cost**. What problems/limitations do you encounter in comparison with simulated annealing?

<div class="alert alert-block alert-info"> <b>Note 1 (Hint)</b>:
    
 One could read the following references: 
 
- [Quantum Optimization: Potential, Challenges, and the Path Forward](https://arxiv.org/abs/2312.02279) and references within.
  
</div>

<div class="alert alert-block alert-info"> <b>Note 2 (Hint)</b>:
    
One can take a look at the [IBM Quantum Development Roadmap to 2033](https://newsroom.ibm.com/2023-12-04-IBM-Debuts-Next-Generation-Quantum-Processor-IBM-Quantum-System-Two,-Extends-Roadmap-to-Advance-Era-of-Quantum-Utility) and [QuEra's Quantum Computing Roadmap](https://www.quera.com/press-releases/quera-computing-releases-a-groundbreaking-roadmap-for-advanced-error-corrected-quantum-computers-pioneering-the-next-frontier-in-quantum-innovation) for an idea about the current and predicted quantum capabilities.
    
</div>

**Answer:**

In [ ]:
# write your code here

### Section 6 (BONUS):

Above you followed specific steps and used simulated annealing as well as quantum or quantum-inspired methods to solve the problem of interest. To do this, you were instructed to incorporate the constraints of the problem (equalities/inequalities) in the objective function and then to find the QUBO formulation of the problem in question. In this exercise you are asked to think of another way to solve the **Mean-Variance-VaR Simplified Version** problem with quantum, quantum-inspired or hybrid methods, without following any of the steps mentioned in this challenge.

**Answer:**

In [ ]:
# write your code here

### Section 7: Pitch your quantum strategy to a client

Imagine that you are part of the Quantum team at Moody's. You are meeting the board of stakeholders of an important company in the US. Given what you now know about the current quantum hardware capabilities, in which quantum algorithms should the stakeholders invest and why? Prepare your pitch focusing only on portfolio optimization problems. How do you think quantum could potentially avoid the drawbacks of classical optimization algorithms and why should a company invest in quantum today?

<div class="alert alert-block alert-info"> <b>Hint</b>:
    
 One could read the following references: 
 
- [Quantum Optimization: Potential, Challenges, and the Path Forward](https://arxiv.org/abs/2312.02279) 
- [QAOA with N⋅p≥200](https://arxiv.org/abs/2303.02064) 
- [Evidence of Scaling Advantage for the Quantum Approximate Optimization Algorithm on a Classically Intractable Problem](https://arxiv.org/pdf/2308.02342.pdf)
- [Towards optimization under uncertainty for fundamental models in energy markets using quantum computers](https://arxiv.org/abs/2301.01108)
  
</div>

**Answer:**

# This is the end of the challenge. Good luck!